In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
output_dir = "graficos_suelo_edificio"
os.makedirs(output_dir, exist_ok=True)

In [4]:
nepal = pd.read_csv("eq2015.csv")
fuente_1 = "https://www.kaggle.com/datasets/sanskarnegi/nepal-earthquake-2015"

In [5]:
distritos = nepal["district_name"].unique()
distritos

array(['OKHALDHUNGA', 'SINDHULI', 'RAMECHHAP', 'DOLAKHA', 'SINDHUPALCHOK',
       'KAVRE', 'NUWAKOT', 'RASUWA', 'DHADING', 'MAKWANPUR', 'GORKHA'],
      dtype=object)

In [6]:
graficos_por_distrito = {}

In [7]:
for distrito in distritos:
    df_distrito = nepal[nepal["district_name"] == distrito]
    df_grouped = df_distrito.groupby(["ground_floor_type", "damage_grade"]).size().unstack(fill_value=0)
    
    # Asegurar presencia de todos los grados de daño
    todos_los_grados = ["Grade 1", "Grade 2", "Grade 3", "Grade 4", "Grade 5"]
    df_grouped = df_grouped.reindex(columns=todos_los_grados, fill_value=0)

    # Normalizar las filas para que cada barra tenga altura total 1
    df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0).fillna(0)

    # Crear la figura
    fig, ax = plt.subplots(figsize=(10, 10))
    df_normalized.plot(kind='bar', stacked=True, colormap='viridis', ax=ax,legend=False)

    # Etiquetas
    plt.xlabel('Material usado para construir la planta baja')
    plt.ylabel('Número de edificios')
    plt.title(f'Distrito: {distrito}')
    plt.xticks(rotation=0)
    ax.set_ylim(0, 1)

    for i, (idx, row) in enumerate(df_normalized.iterrows()):
        y_offset = 0
        for damage_grade, value in row.items():
            if value > 0:
                ax.text(
                    i + 0.48,
                    y_offset + value / 2, 
                    f"{value * 100:.1f}%", 
                    ha='right', 
                    va='center',
                    fontsize=7,
                    color='black'
                )
            y_offset += value
    # Guardar la imagen en memoria
    img_path = os.path.join(output_dir, f"{distrito}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    # Guardar ruta de la imagen
    graficos_por_distrito[distrito] = img_path

In [8]:
lista_gorkha = nepal[nepal["district_name"] == "GORKHA"]

In [9]:
from scipy.stats import chi2_contingency, fisher_exact

# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_gorkha[lista_gorkha["ground_floor_type"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["ground_floor_type"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["ground_floor_type"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Mud", "Brick/Stone")
analizar_contraste("Mud", "Other")
analizar_contraste("Mud", "RC")
analizar_contraste("Mud", "Timber")


📊 Comparando: Mud vs Brick/Stone

🔍 Test de Chi-cuadrado:
Chi² = 72.87, dof = 4, p-valor = 5.6215e-15

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.84, p-valor = 1.3753e-10

📊 Comparando: Mud vs Other

🔍 Test de Chi-cuadrado:
Chi² = 95.81, dof = 4, p-valor = 7.6550e-20

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.31, p-valor = 3.7862e-07

📊 Comparando: Mud vs RC

🔍 Test de Chi-cuadrado:
Chi² = 17838.72, dof = 4, p-valor = 0.0000e+00

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.10, p-valor = 0.0000e+00

📊 Comparando: Mud vs Timber

🔍 Test de Chi-cuadrado:
Chi² = 88.75, dof = 4, p-valor = 2.4207e-18

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.17, p-valor = 1.3311e-02


In [11]:
lista_dolakha = nepal[nepal["district_name"] == "DOLAKHA"]

In [12]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_dolakha[lista_dolakha["ground_floor_type"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["ground_floor_type"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["ground_floor_type"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Mud", "Brick/Stone")
analizar_contraste("Mud", "Other")
analizar_contraste("Mud", "RC")
analizar_contraste("Mud", "Timber")


📊 Comparando: Mud vs Brick/Stone

🔍 Test de Chi-cuadrado:
Chi² = 487.07, dof = 4, p-valor = 4.1851e-104

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.89, p-valor = 3.0562e-83

📊 Comparando: Mud vs Other

🔍 Test de Chi-cuadrado:
Chi² = 86.64, dof = 4, p-valor = 6.7979e-18

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.28, p-valor = 3.0323e-05

📊 Comparando: Mud vs RC

🔍 Test de Chi-cuadrado:
Chi² = 11351.06, dof = 4, p-valor = 0.0000e+00

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.10, p-valor = 0.0000e+00

📊 Comparando: Mud vs Timber

🔍 Test de Chi-cuadrado:
Chi² = 420.23, dof = 4, p-valor = 1.1846e-89

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.32, p-valor = 3.7996e-14


In [15]:
lista_makwanpur = nepal[nepal["district_name"] == "MAKWANPUR"]

In [16]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_makwanpur[lista_makwanpur["ground_floor_type"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["ground_floor_type"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["ground_floor_type"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Mud", "Brick/Stone")
analizar_contraste("Mud", "Other")
analizar_contraste("Mud", "RC")
analizar_contraste("Mud", "Timber")


📊 Comparando: Mud vs Brick/Stone

🔍 Test de Chi-cuadrado:
Chi² = 1405.66, dof = 4, p-valor = 4.0942e-303

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 2.34, p-valor = 1.0284e-61

📊 Comparando: Mud vs Other

🔍 Test de Chi-cuadrado:
Chi² = 130.15, dof = 4, p-valor = 3.6124e-27

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.07, p-valor = 1.0764e-14

📊 Comparando: Mud vs RC

🔍 Test de Chi-cuadrado:
Chi² = 11496.03, dof = 4, p-valor = 0.0000e+00

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.11, p-valor = 0.0000e+00

📊 Comparando: Mud vs Timber

🔍 Test de Chi-cuadrado:
Chi² = 338.29, dof = 4, p-valor = 5.9133e-72

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.32, p-valor = 1.3091e-08


In [18]:
lista_rasuwa = nepal[nepal["district_name"] == "RASUWA"]

In [19]:
# Función para hacer el análisis entre dos tipos de adosamiento
def analizar_contraste(position_a, position_b):
    print(f"\n📊 Comparando: {position_a} vs {position_b}")
    
    # Filtrar los dos grupos
    subset = lista_rasuwa[lista_rasuwa["ground_floor_type"].isin([position_a, position_b])].copy()

    # ------------------------------
    # 1. Test de Chi-cuadrado completo
    # ------------------------------
    tabla_chi = pd.crosstab(subset["ground_floor_type"], subset["damage_grade"])
    chi2, p_chi2, dof, expected = chi2_contingency(tabla_chi)
    
    print(f"\n🔍 Test de Chi-cuadrado:")
    print(f"Chi² = {chi2:.2f}, dof = {dof}, p-valor = {p_chi2:.4e}")
    
    # ------------------------------
    # 2. Test exacto de Fisher (colapsado vs no colapsado)
    # ------------------------------
    subset["colapsado"] = subset["damage_grade"] == "Grade 5"
    tabla_fisher = pd.crosstab(subset["ground_floor_type"], subset["colapsado"])
    
    # Solo se puede aplicar si la tabla es 2x2
    if tabla_fisher.shape == (2, 2):
        oddsratio, p_fisher = fisher_exact(tabla_fisher)
        print(f"\n🔬 Test exacto de Fisher (Grade 5 vs resto):")
        print(f"Odds Ratio = {oddsratio:.2f}, p-valor = {p_fisher:.4e}")
    else:
        print("⚠️ No se puede aplicar test de Fisher (tabla no 2x2).")

# Comparaciones deseadas
analizar_contraste("Mud", "Brick/Stone")
analizar_contraste("Mud", "Other")
analizar_contraste("Mud", "RC")
analizar_contraste("Mud", "Timber")


📊 Comparando: Mud vs Brick/Stone

🔍 Test de Chi-cuadrado:
Chi² = 86.57, dof = 4, p-valor = 7.0540e-18

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 1.30, p-valor = 5.3144e-06

📊 Comparando: Mud vs Other

🔍 Test de Chi-cuadrado:
Chi² = 162.51, dof = 4, p-valor = 4.2362e-34

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.00, p-valor = 5.2791e-02

📊 Comparando: Mud vs RC

🔍 Test de Chi-cuadrado:
Chi² = 3155.47, dof = 4, p-valor = 0.0000e+00

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.09, p-valor = 7.3961e-182

📊 Comparando: Mud vs Timber

🔍 Test de Chi-cuadrado:
Chi² = 176.05, dof = 4, p-valor = 5.2648e-37

🔬 Test exacto de Fisher (Grade 5 vs resto):
Odds Ratio = 0.33, p-valor = 2.2672e-23


In [20]:
# Filtrar colapsados y no colapsados
colapsado = lista_rasuwa[lista_rasuwa["damage_grade"] == "Grade 5"]
no_colapsado = lista_rasuwa[lista_rasuwa["damage_grade"].isin(["Grade 1", "Grade 2", "Grade 3", "Grade 4"])]

# Contar por tipo de adosamiento
colapsado_counts = colapsado["ground_floor_type"].value_counts()
no_colapsado_counts = no_colapsado["ground_floor_type"].value_counts()

# Unir los conteos
tabla_riesgo = pd.DataFrame({
    "Colapsó (Grade 5)": colapsado_counts,
    "No colapsó (Grade 1–4)": no_colapsado_counts
})
tabla_riesgo["Total"] = tabla_riesgo["Colapsó (Grade 5)"] + tabla_riesgo["No colapsó (Grade 1–4)"]

# Rellenar con 0 si hay posiciones que faltan en uno de los dos
tabla_riesgo = tabla_riesgo.fillna(0).astype(int)

# Mostrar
print(tabla_riesgo)

                   Colapsó (Grade 5)  No colapsó (Grade 1–4)  Total
ground_floor_type                                                  
Brick/Stone                     1341                     520   1861
Mud                             7474                    2228   9702
Other                              0                       2      0
RC                               170                     543    713
Timber                           193                     173    366
